# Day 21: Allergen Assessment

[*Advent of Code 2020 day 21*](https://adventofcode.com/2020/day/21) and [*solution megathread*](https://redd.it/khaiyk)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2020/21/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2020%2F21%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

In [5]:
testdata = """mxmxvkd kfcds sqjhc nhms (contains dairy, fish)
trh fvjkl sbzzf mxmxvkd (contains dairy)
sqjhc fvjkl (contains soy)
sqjhc mxmxvkd sbzzf (contains fish)""".splitlines()

inputdata = downloaded['input'].splitlines()

In [6]:
from functools import reduce


def parse(lines):
    contents = []
    for line in lines:
        ingredients, allergens = line[:-1].split(' (contains ')
        contents.append((set(ingredients.split()), set(allergens.split(', '))))
    return contents


def munge(contents):
    allergen_candidates = dict()
    for ingredients, allergens in contents:
        for allergen in allergens:
            if allergen not in allergen_candidates:
                allergen_candidates[allergen] = [ingredients]
            else:
                allergen_candidates[allergen].append(ingredients)
    return allergen_candidates


def reduce_candidates(allergen_present_ins, debug=False):
    allergen_present_in = dict()
    for allergen, candidatess in allergen_present_ins.items():
        allergen_present_in[allergen] = reduce(lambda a, b: a & b, candidatess)
        if debug:
            print(f'allergen: {allergen}, '
                  f'candidates: {allergen_present_in[allergen]}')
    return allergen_present_in


def eliminate_candidate(allergen_present_in, eliminated):
    for allergen, candidates in allergen_present_in.items():
        if eliminated in candidates:
            allergen_present_in[allergen].remove(eliminated)


def deduce_candidates(allergen_present_in, debug=False):
    eliminated = dict()
    for allergen, candidates in allergen_present_in.copy().items():
        if len(candidates) == 1:
            eliminated[allergen] = candidates.pop()
            eliminate_candidate(allergen_present_in, eliminated[allergen])
            allergen_present_in.pop(allergen)
            if debug:
                print(f'Deduced that {eliminated[allergen]} '
                      f'must be {allergen}!')
    return eliminated


def count_unknown(contents, known):
    counter = 0
    known_ingredients = set(known.values())
    for ingredients, _ in contents:
        counter += len(ingredients - known_ingredients)
    return counter

In [7]:
contents = parse(testdata)
allergen_present_ins = munge(contents)
allergen_present_in = reduce_candidates(allergen_present_ins, debug=True)
ingredients = dict()
finished = False
while not finished:
    eliminated = deduce_candidates(allergen_present_in, debug=True)
    if len(eliminated) == 0:
        finished = True
    else:
        ingredients.update(eliminated)
unknown_occurences = count_unknown(contents, ingredients)
print(unknown_occurences)

allergen: dairy, candidates: {'mxmxvkd'}
allergen: fish, candidates: {'mxmxvkd', 'sqjhc'}
allergen: soy, candidates: {'fvjkl', 'sqjhc'}
Deduced that mxmxvkd must be dairy!
Deduced that sqjhc must be fish!
Deduced that fvjkl must be soy!
5


In [8]:
contents = parse(inputdata)
allergen_present_ins = munge(contents)
allergen_present_in = reduce_candidates(allergen_present_ins)
ingredients = dict()
finished = False
while not finished:
    eliminated = deduce_candidates(allergen_present_in)
    if len(eliminated) == 0:
        finished = True
    else:
        ingredients.update(eliminated)
unknown_occurences = count_unknown(contents, ingredients)
print(unknown_occurences)

1958


In [9]:
HTML(downloaded['part1_footer'])

## Part Two

In [10]:
HTML(downloaded['part2'])

In [11]:
','.join([ingrendient
          for allergen, ingrendient
          in sorted(ingredients.items(),
                    key=lambda item: item[0])])

'xxscc,mjmqst,gzxnc,vvqj,trnnvn,gbcjqbm,dllbjr,nckqzsg'

In [12]:
HTML(downloaded['part2_footer'])